In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
import os
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import pathlib
import time
import IPython.display as display
import random
from tensorflow import keras
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from google.colab import drive
from google.colab.patches import cv2_imshow
import cv2
from PIL import Image



In [105]:
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
 def timer(start,end):
    hours, rem = divmod(end - start, 3600)
    minutes, seconds = divmod(rem, 60)
    result = str("{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours),int(minutes),seconds))
    return result 


Start_time_Scrpit1 = time.time()

In [0]:

h_pixels = w_pixels = 100


# preprocesar imatges normals
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels = 1)             
  image = tf.image.resize_images(image, [h_pixels, w_pixels])
  image /= 255.0  # normalitzem al rang [0, 1]
  
  return image




def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
    
  return preprocess_image(image)

In [0]:
os.chdir('/content/gdrive/My Drive/Data_Aug/')  #change dir

# !mkdir train
# !mkdir test

In [0]:
# !unzip train.zip -d train/
# !unzip test.zip -d test/ 

In [0]:

def Mirroring_Data_Generator(IP):
  
  for x in range(len(IP)): 
    im_path = IP[x]
 
    if(str.find(im_path, "train/PNEUMONIA") > 0):
      print(str(x + 1) + "/5211.\t train/PNEUMONIA")   
      mat = cv2.flip(cv2.imread(im_path), 1)     # 1 = horitzontal
      im = Image.fromarray(mat)
      im.save("/content/gdrive/My Drive/Data_Aug/train/train/PNEUMONIA/data_aug" + str(x)  + ".jpeg")

    elif(str.find(im_path, "train/NORMAL") > 0):
      print(str(x + 1) + "/5211.\t train/NORMAL")
      mat = cv2.flip(cv2.imread(im_path), 1)
      im = Image.fromarray(mat)
      im.save("/content/gdrive/My Drive/Data_Aug/train/train/NORMAL/data_aug" + str(x)  + ".jpeg")

    elif(str.find(im_path, "test/PNEUMONIA") > 0):
      print(str(x + 1) + "/624.\t test/PNEUMONIA")
      mat = cv2.flip(cv2.imread(im_path), 1)
      im = Image.fromarray(mat)
      im.save("/content/gdrive/My Drive/Data_Aug/test/test/PNEUMONIA/data_aug" + str(x)  + ".jpeg")

    elif(str.find(im_path, "test/NORMAL") > 0):
      print(str(x + 1) + "/624.\t test/NORMAL")
      mat = cv2.flip(cv2.imread(im_path), 1)
      im = Image.fromarray(mat)
      im.save("/content/gdrive/My Drive/Data_Aug/test/test/NORMAL/data_aug_" + str(x)  + ".jpeg")

    else:
      None
     

In [0]:
#Mirroring_Data_Generator(train_image_paths)
#Mirroring_Data_Generator(test_image_paths)

In [111]:

DirectoriDades_aug = pathlib.Path('/content/gdrive/My Drive/Data_Aug')
train_dir_aug = DirectoriDades_aug / 'train' / 'train/'
test_dir_aug  = DirectoriDades_aug / 'test'/ 'test/'


print("Train dir: {}".format(train_dir_aug))
print("Test dir: {}".format(test_dir_aug))



os.chdir(pathlib.Path(train_dir_aug))
train_image_paths_aug = list(train_dir_aug.glob('*/*.jpeg'))
train_image_paths_aug = [str(path) for path in train_image_paths_aug]

random.shuffle(train_image_paths_aug)
train_image_count_aug = len(train_image_paths_aug)



os.chdir(test_dir_aug)
test_image_paths_aug = list(test_dir_aug.glob('*/*.jpeg'))
test_image_paths_aug = [str(path) for path in test_image_paths_aug]

random.shuffle(test_image_paths_aug)
test_image_count_aug = len(test_image_paths_aug)


print("Primera imatge Train:\t{}".format(train_image_paths_aug[0]))
print("Primera imatge Test:\t{}".format(test_image_paths_aug[0]))
print("\n")
print("Nombre d'imatges Train:\t{}".format(train_image_count_aug))
print("Nombre d'imatges Test:\t{}".format(test_image_count_aug))

Train dir: /content/gdrive/My Drive/Data_Aug/train/train
Test dir: /content/gdrive/My Drive/Data_Aug/test/test
Primera imatge Train:	/content/gdrive/My Drive/Data_Aug/train/train/PNEUMONIA/data_aug211.jpeg
Primera imatge Test:	/content/gdrive/My Drive/Data_Aug/test/test/PNEUMONIA/data_aug186.jpeg


Nombre d'imatges Train:	10422
Nombre d'imatges Test:	1248


## Funció Data Augemtation

In [0]:
def Define_model(capes, epoques, h_pixels, w_pixels, DropOut, NumMod):


                                                                                      
  
  print('\n\n\n\n\n\n#################################################################')
  print('\t\t MODEL:\t\t'+ str(NumMod))
  print('\t\t CAPES:\t\t'+ str(capes))
  print('\t\t EPOQUES:\t'+ str(epoques))
  print('\t\t PIXELS:\t(' + str(h_pixels) + ',' +  str(w_pixels)+ ')')
  print('#################################################################')
  
  
  callbacks = [keras.callbacks.EarlyStopping( monitor = 'val_loss',
                                              min_delta = 1e-2,
                                              patience = 2,
                                              verbose = 1)]

  start_time = time.time()
  
  train_roc_auc_list = []
  test_roc_auc_list  = []
  
  
  
  
  Percent_Data = np.arange(0.1, 1.1, 0.1)
  iteration = 1  
  
  for i in Percent_Data:


    
    
    train_image_paths_aug2  = random.choices(train_image_paths_aug, k = int(train_image_count_aug * i))    # Afame el i% de les imatges train 
    test_image_paths_aug2   = random.choices(test_image_paths_aug, k = int(test_image_count_aug * i))       # Afame el i% de les imatges test

    label_names_aug = sorted(item.name for item in train_dir_aug.glob('*/') if item.is_dir())
    label_to_index_aug = dict((name, index) for index, name in enumerate(label_names_aug))


    train_image_labels_aug = [label_to_index_aug[pathlib.Path(path).parent.name] for path in train_image_paths_aug2] 
    test_image_labels_aug  = [label_to_index_aug[pathlib.Path(path).parent.name] for path in test_image_paths_aug2 ]
    
    
    
    auto = tf.data.experimental.AUTOTUNE


    train_path_ds_aug = tf.data.Dataset.from_tensor_slices(train_image_paths_aug2)
    test_path_ds_aug  = tf.data.Dataset.from_tensor_slices(test_image_paths_aug2)
    
    
    train_image_ds_aug = train_path_ds_aug.map(load_and_preprocess_image, num_parallel_calls = auto)
    test_image_ds_aug  = test_path_ds_aug.map(load_and_preprocess_image, num_parallel_calls = auto)

  
    train_label_ds_aug = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels_aug, tf.int64))
    test_label_ds_aug  = tf.data.Dataset.from_tensor_slices(tf.cast(test_image_labels_aug, tf.int64))

    train_image_label_ds_aug = tf.data.Dataset.zip((train_image_ds_aug, train_label_ds_aug))
    test_image_label_ds_aug  = tf.data.Dataset.zip((test_image_ds_aug, test_label_ds_aug))

    
    batch_size = 16

    train_ds = train_image_label_ds_aug.shuffle(buffer_size = train_image_count_aug)
    train_ds = train_ds.repeat()
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.prefetch(buffer_size = batch_size)

    train_ds = test_image_label_ds_aug.shuffle(buffer_size = test_image_count_aug)
    test_ds = test_image_label_ds_aug.repeat()
    test_ds = test_ds.batch(batch_size)
    test_ds = test_ds.prefetch(buffer_size=16)
    
    
    
    
    model = tf.keras.Sequential()



    if(capes == 5): 
      print("\nCapes 5: (32, 32) + (64, 64) + FDD")        
      model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding="same", input_shape=(h_pixels, w_pixels, 1)))
      model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
      #model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

      model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
      model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
      m#odel.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

      if(DropOut == 'Yes'):
        model.add(tf.keras.layers.Dropout(0.5))
      else:
        None

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(256, activation='relu'))
      model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



    elif(capes == 6): 
      print("\nCapes 6: (32, 32) + (64, 64) + (128) + FDD")        
      model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding="same", input_shape=(h_pixels, w_pixels, 1)))
      model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
      #model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

      model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
      model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
      #model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

      model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))

      if(DropOut == 'Yes'):
        model.add(tf.keras.layers.Dropout(0.5))
      else:
        None
      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(256, activation='relu'))
      model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


    model.compile(optimizer = "adam", 
                  loss = "binary_crossentropy", 
                  metrics = ["accuracy"])


     
    if(iteration == 1):
      model.summary()    
    else:
      None


    
    



    train_steps_per_epoch = len(train_image_paths_aug) / batch_size
    test_steps_per_epoch  = len(test_image_paths_aug)  / batch_size



    print('\n\n=================================================================') 
    print('      E N T R E N A M E N T     D E L      M O D E L     ' + str(NumMod))
    print("                  -----------------------")
    print("                      Iteració  {} / {}".format(iteration, len(Percent_Data)))
    print('=================================================================\n')



    model.fit(train_ds,
              steps_per_epoch = int(train_steps_per_epoch), 
              validation_data = test_ds, 
              validation_steps = int(test_steps_per_epoch),
              callbacks = callbacks,
              epochs = epoques)



    y_train_pred = model.predict(train_image_label_ds_aug.batch(batch_size), steps=int(train_steps_per_epoch))
    y_train = np.array(train_image_labels_aug)

    y_test_pred = model.predict(test_image_label_ds_aug, steps=int(test_steps_per_epoch))
    y_test = np.array(test_image_labels_aug)

    print(len(y_train_pred))
    print(len(y_train))
    print(len(y_test_pred))
    print(len(y_test))



    train_fpr, train_tpr, _ = roc_curve(y_train, y_train_pred)
    train_roc_auc = auc(train_fpr, train_tpr)

    test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred)
    test_roc_auc = auc(test_fpr, test_tpr)
    
    train_roc_auc_list.append(train_roc_auc)
    test_roc_auc_list.append(test_roc_auc)
    
    print('AUC train vect:' + str(train_roc_auc_list))
    print('AUC test vect:'  + str(test_roc_auc_list))
    
    print('AUC train :' + str(train_roc_auc))
    print('AUC test :'  + str(test_roc_auc))
  
    iteration = iteration + 1
   
  
  runtime_model = timer(start_time, time.time())
  print("\nTemps d'execució: %s" % runtime_model)
  
  globals().update(locals())

  
  return  train_roc_auc_list, test_roc_auc_list, runtime_model




In [125]:


h_pixels = w_pixels = 100
Epoques = 25
Capes = np.arange(2, 10, 1)
n = len(Capes)

NumModel =  np.arange(1, n +1, 1)


DropOut = 'False'
DataAugmentation = 'False'

Model1 = []
for index in range(n):
    model1 = 'Model_' + str(index + 1)
    Model1.append(model1)
    
    

        
    

df_AucRoc = { 'Model': Model1, 
              'Capes': Capes, 
              'Epoques': Epoques,
              'Pxls_Fil_Col': h_pixels,
              'DropOut': DropOut,
              'NumMod': NumModel,}


Param_AUCROC_Conv  = pd.DataFrame(data = df_AucRoc)
Param_AUCROC_Conv =  Param_AUCROC_Conv.iloc[4:5]
Param_AUCROC_Conv

,Model,Capes,Epoques,Pxls_Fil_Col,DropOut,NumMod
4,Model_5,6,25,100,False,5


In [126]:
for i, row in Param_AUCROC_Conv.iterrows():
  
  Model = row['Model']
  Capes = row['Capes']
  Epoques = row['Epoques']
  Pixels = row['Pxls_Fil_Col']
  DropOut = row['DropOut']
  NumMod = row['NumMod']
  
  
  auc_train_5, auc_test_5, runtime_model_5 = Define_model( capes =  Capes, 
                                                           epoques = Epoques, 
                                                           h_pixels = Pixels,  
                                                           w_pixels = Pixels,
                                                           DropOut = DropOut,
                                                           NumMod = NumMod)


   
   
  
  #Eval_AUCROC_Conv_aug.to_csv('/content/gdrive/My Drive/Resultats/Param_Definicio_Model_CAPES_DATA_AUGMENTATION.csv', index = False, header = True)







#################################################################
		 MODEL:		5
		 CAPES:		6
		 EPOQUES:	25
		 PIXELS:	(100,100)
#################################################################

Capes 6: (32, 32) + (64, 64) + (128) + FDD
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 100, 100, 32)      320       
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 98, 98, 32)        9248      
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 94, 94, 64)        36928     
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 92, 92, 128)       73856     
______________________________

ValueError: ignored